In [1]:
from dotenv import load_dotenv
from app import ChatBot, ChatBotConfig
from typing import get_args

/Users/tmskss/Development/rag-chatbot-demo/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
from langfuse import Langfuse

langfuse = Langfuse(
  secret_key="sk-lf-5661f1bf-a313-4291-a5ee-a40acf2f6073",
  public_key="pk-lf-2e5389e9-580f-482e-bff5-c6845259eac1",
  host="http://localhost:3000"
)

In [4]:
from langfuse.callback import CallbackHandler

trace = {
        "callbacks": [
            CallbackHandler(
                secret_key="sk-lf-5661f1bf-a313-4291-a5ee-a40acf2f6073",
                public_key="pk-lf-2e5389e9-580f-482e-bff5-c6845259eac1",
                host="http://localhost:3000",
            )
        ]
    }

In [5]:
import uuid

In [6]:
import random
import gradio as gr

username = "Ericsson"

modelfamilies_model_dict = {
    "GPT": get_args(ChatBotConfig.OPENAI_MODELS),
    "Mistral": get_args(ChatBotConfig.MISTRAL_MODELS),
    "Llama": get_args(ChatBotConfig.LLAMA_MODELS),
}

system_prompt = []
temperature = []
max_tokens = []
ChatBot.store = {}

def exec_prompt(chatbot, question, session_id, model_family = "Mistral", model="mistral-large"):

    question = question
    # get chain
    chain = ChatBot.get_chain(model_family=model_family, model=model)
    response = chain.invoke({"question": question}, config={"configurable": {"user_id": username, "conversation_id": session_id}} | trace)
    chat_history = ChatBot.get_session_history(username, session_id)

    history_pairs = []
    for msg in chat_history.messages:
        if msg.type == "human":
            history_pairs.append([msg.content, ""])
        elif msg.type == "ai":
            history_pairs[-1][1] = msg.content

    return history_pairs, "" 

def exec_prompt_streaming(chatbot, question, session_id, model_family = "Mistral", model="mistral-large"):

    question = question
    # get chain
    chain = ChatBot.get_chain(model_family=model_family, model=model)
    chat_history = ChatBot.get_session_history(username, session_id)
    response = chain.stream({"question": question}, config={"configurable": {"user_id": username, "conversation_id": session_id}}| trace)

    history_pairs = []
    for msg in chat_history.messages:
        if msg.type == "human":
            history_pairs.append([msg.content, ""])
        elif msg.type == "ai":
            history_pairs[-1][1] = msg.content

    history_pairs.append([question, ""])
    for res in response:
        if res is not None:
            history_pairs[-1][1] += res
        yield history_pairs, ""         

gr.close_all()

callback = gr.CSVLogger()

with gr.Blocks(title="CompSoft") as demo:
    session_id = gr.Textbox(value = uuid.uuid4, interactive=False, visible=False)
    gr.Markdown("# Component Soft 5G RAG Demo")
    #system_prompt = gr.Textbox(label="System prompt", value="You are a helpful, harmless and honest assistant.")
    with gr.Row():
        modelfamily = gr.Dropdown(list(modelfamilies_model_dict.keys()), label="Model family", value="GPT")
        model = gr.Dropdown(list(modelfamilies_model_dict["GPT"]), label="Model", value="gpt-3.5-turbo")       
        """temperature = gr.Slider(label="Temperature:", minimum=0, maximum=2, value=1,
            info="LLM generation temperature")
        max_tokens = gr.Slider(label="Max tokens", minimum=100, maximum=2000, value=500, 
            info="Maximum number of generated tokens")"""
    with gr.Row():
        chatbot=gr.Chatbot(label="CompSoft_5G_RAG", height=400, show_copy_button=True)
    with gr.Row():
        prompt = gr.Textbox(label="Question", value="What is 5G?")
    with gr.Row():
        submit_btn_nostreaming = gr.Button("Answer")
        submit_btn_streaming = gr.Button("Answer with streaming")
        clear_btn = gr.ClearButton([prompt, chatbot])
        #flag_btn = gr.Button("Flag")
    
    
    @modelfamily.change(inputs=modelfamily, outputs=[model])
    def update_modelfamily(modelfamily):
        model = list(modelfamilies_model_dict[modelfamily])
        return gr.Dropdown(choices=model, value=model[0], interactive=True)

    submit_btn_streaming.click(exec_prompt_streaming, inputs=[chatbot, prompt, session_id, modelfamily, model], outputs=[chatbot, prompt])
    submit_btn_nostreaming.click(exec_prompt, inputs=[chatbot, prompt, session_id, modelfamily, model], outputs=[chatbot, prompt])
    clear_btn.click(lambda session_id: ChatBot.del_session_history(username, session_id), [session_id], None, preprocess=False)

    #callback.setup([system_prompt, modelfamily, model, temperature, max_tokens, chatbot], "flagged_data_points")
    #flag_btn.click(lambda *args: callback.flag(args), [system_prompt, modelfamily, model, temperature, max_tokens, chatbot], None, preprocess=False)
    
    gr.Examples(
        ["What is 5G?", "What are the main adventages of 5G compared to 4G?", "What frequencies does 5G use?", "What is OFDMA?", 
         "Which organisations are responsible for the standardization of 5G?"],
        prompt
    )

#demo.launch()
demo.launch(share=True, share_server_address="gradio.componentsoft.ai:7000", share_server_protocol="https", auth=("Ericsson", "Torshamnsgatan21"), max_threads=20, show_error=True, state_session_capacity=20)
#demo.launch(share=True, share_server_address="gradio.componentsoft.ai:7000", share_server_protocol="https", auth=("Ericsson", "Torshamnsgatan21"), max_threads=20, show_error=True, favicon_path="/home/rconsole/GIT/AI-434/source/labfiles/data/favicon.ico", state_session_capacity=20)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ae815ff8c221df64a9.gradio.componentsoft.ai

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/Users/tmskss/Development/rag-chatbot-demo/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
